In [26]:
### IMPORTS
#
from flask import Flask, jsonify, render_template, request
from pandas.plotting import lag_plot
from sklearn.metrics import mean_squared_error
from sqlalchemy import create_engine
from sqlite3 import Error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
# import statsmodels.tsa.arima.model.ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
# from statsmodels.tsa.arima.model import ARIMAResults
from statsmodels.tsa.stattools import adfuller
import dataframe_image as dfi
import datetime
import glob
import joblib
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import sqlite3
import time
# END IMPORTS

In [27]:
database = r'crypto_db.sqlite'

def create_connection(db_file):
    """ create a database connection to the SQLite database specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn
# END create_connection


### RUN SQL COMMAND
#
def execute_sql_cmd(conn, command):
    """ run a sql command statement
    :param conn: Connection object
    :param execute_sql_cmd: run sql statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(command)
    except Error as e:
        print(e)
# END execute_sql_cmd


### LOAD SCHEMA
#
conn = create_connection(database)
# END LOAD SCHEMA

In [31]:
from datetime import datetime, timedelta

asset = 'DOGE'

path = f'models/{asset}/'

MODEL = joblib.load(f'{path}{asset}_model.smd')

numdays = 365

conn = create_connection(database)
query = f"SELECT time_period_end, price_close FROM historic_data WHERE asset_id='{asset}' ORDER BY time_period_end ASC"
if conn is not None:
    result = pd.read_sql_query(query,conn)
else:
    print("Error! cannot create the database connection.")
conn.close()

result['time_period_end'] = pd.to_datetime(result['time_period_end'])

plotstart = result['time_period_end'].iloc[-1]

date = plotstart

graph = pd.DataFrame({'date' : [plotstart]})

for i in range(numdays - 1):
    date += timedelta(days=1)
    graph = graph.append({'date' : date}, ignore_index=True)      



forecast = MODEL.forecast(numdays)[0]
#MODEL.forecast() shows predicted value, std error of forecast, and confidence interval (lower & upper bounds)

graph['forecast'] = forecast

graph['forecast'][0] = result['price_close'].iloc[-1].copy()

display(result)
display(graph)

#### PLOTLY EXAMPLE
##


import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure([

    go.Scatter(
        name='Actual',
        x=result.time_period_end,
        y=result.price_close,
        mode='lines',
        marker=dict(color="#008080"),
        line=dict(width=1),
        showlegend=True
    ),
    go.Scatter(
        name='Forecast',
        x=graph.date,
        y=graph.forecast,
        marker=dict(color="#FF8C00"),
        line=dict(width=1),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        showlegend=True
    )
])

fig.update_layout(
    yaxis_title='',
    title='',
    hovermode="x"
)

fig.show()

C:\Users\jakeb\anaconda3\envs\data_analytics\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning:


statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)


<ipython-input-31-c97946135e22>:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

,time_period_end,price_close
0,2014-02-25 00:00:00+00:00,1.149425e-03
1,2014-02-28 00:00:00+00:00,1.000000e-07
2,2014-03-04 00:00:00+00:00,1.000000e-03
3,2014-03-06 00:00:00+00:00,1.379310e-03
4,2014-03-07 00:00:00+00:00,1.379310e-03
...,...,...
467,2021-02-21 00:00:00+00:00,5.423050e-02
468,2021-02-22 00:00:00+00:00,5.607330e-02
469,2021-02-23 00:00:00+00:00,5.368370e-02
470,2021-02-24 00:00:00+00:00,4.763940e-02


,date,forecast
0,2021-02-25 00:00:00+00:00,0.055810
1,2021-02-26 00:00:00+00:00,0.050980
2,2021-02-27 00:00:00+00:00,0.048838
3,2021-02-28 00:00:00+00:00,0.049873
4,2021-03-01 00:00:00+00:00,0.051268
...,...,...
360,2022-02-20 00:00:00+00:00,0.085811
361,2022-02-21 00:00:00+00:00,0.085912
362,2022-02-22 00:00:00+00:00,0.086014
363,2022-02-23 00:00:00+00:00,0.086116
